In [1]:
!pip install git+https://github.com/kareem1925/pennylane-qulacs@GPU_support

  Cloning https://github.com/kareem1925/pennylane-qulacs (to revision GPU_support) to /tmp/pip-req-build-e91pfoda
  Running command git clone -q https://github.com/kareem1925/pennylane-qulacs /tmp/pip-req-build-e91pfoda
  Running command git checkout -b GPU_support --track origin/GPU_support
  Switched to a new branch 'GPU_support'
  Branch 'GPU_support' set up to track remote branch 'GPU_support' from 'origin'.
     |████████████████████████████████| 163kB 3.4MB/s 
     |████████████████████████████████| 276kB 9.0MB/s 
  Created wheel for pennylane-qulacs: filename=pennylane_qulacs-0.0.4-cp36-none-any.whl size=4564 sha256=29f57de6b1b035552950e72006ea9bdd663e87f045b04350e9dc97d29b99b2fc
  Stored in directory: /tmp/pip-ephem-wheel-cache-tfvt2amk/wheels/93/58/04/19f31c7d05f27562d0941ae63f8c34e73620017d13840d4d00
  Created wheel for qulacs-gpu: filename=Qulacs_GPU-0.1.9-cp36-cp36m-linux_x86_64.whl size=839889 sha256=bdc4814b28022d246c749d19ab34b3806dcf7c3ae890962a997c7c419ea3f572
  Stored

In [0]:
import os 
os.kill(os.getpid(), 9)

In [1]:
import qulacs
qulacs.QuantumStateGpu

qulacs.QuantumStateGpu

In [0]:
from pennylane import numpy as np
from sklearn.preprocessing import LabelBinarizer

def _weighted_sum(sample_score, sample_weight, normalize=False):
    if normalize:
        return np.average(sample_score, weights=sample_weight)
    elif sample_weight is not None:
        return np.dot(sample_score, sample_weight)
    else:
        return sample_score.sum()


def log_loss(y_true, y_pred, eps=1e-15, normalize=True, sample_weight=None,
             labels=None):
    

    lb = LabelBinarizer()

    if labels is not None:
        lb.fit(labels)
    else:
        lb.fit(y_true)

    if len(lb.classes_) == 1:
        if labels is None:
            raise ValueError('y_true contains only one label ({0}). Please '
                             'provide the true labels explicitly through the '
                             'labels argument.'.format(lb.classes_[0]))
        else:
            raise ValueError('The labels array needs to contain at least two '
                             'labels for log_loss, '
                             'got {0}.'.format(lb.classes_))

    transformed_labels = lb.transform(y_true)

    if transformed_labels.shape[1] == 1:
        transformed_labels = np.append(1 - transformed_labels,
                                       transformed_labels, axis=1)

    # Clipping
    y_pred = np.clip(y_pred, eps, 1 - eps)

    # If y_pred is of single dimension, assume y_true to be binary
    # and then check.
    if y_pred.ndim == 1:
        y_pred = y_pred[:, np.newaxis]
    if y_pred.shape[1] == 1:
        y_pred = np.append(1 - y_pred, y_pred, axis=1)

    # Check if dimensions are consistent.
#    transformed_labels = check_array(transformed_labels)
    if len(lb.classes_) != y_pred.shape[1]:
        if labels is None:
            raise ValueError("y_true and y_pred contain different number of "
                             "classes {0}, {1}. Please provide the true "
                             "labels explicitly through the labels argument. "
                             "Classes found in "
                             "y_true: {2}".format(transformed_labels.shape[1],
                                                  y_pred.shape[1],
                                                  lb.classes_))
        else:
            raise ValueError('The number of classes in labels is different '
                             'from that in y_pred. Classes found in '
                             'labels: {0}'.format(lb.classes_))

    # Renormalize
    y_pred /= y_pred.sum(axis=1)[:, np.newaxis]
    loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
#    print(loss)

    return loss
from sklearn.metrics import balanced_accuracy_score as acc

def accuracy_score(y_true, y_pred):
    """Accuracy score.

    Args:
        y_true (array[float]): 1-d array of targets
        y_predicted (array[float]): 1-d array of predictions
        state_labels (array[float]): 1-d array of state representations for labels

    Returns:
        score (float): the fraction of correctly classified samples
    """
    weights = compute_sample_weight('balanced',y_true)

    return acc(y_true,y_pred,sample_weight=weights)


In [0]:
from sklearn.model_selection import train_test_split
from pennylane import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import compute_sample_weight
import pennylane as qml
from sklearn.datasets import load_digits

In [53]:
X,y = load_digits(n_class=2,return_X_y=True)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=5,stratify=y)
y_train = np.array([-1 if i==0 else i for i in y_train])
#y_train = np.array([0 if i==1 else i for i in y_train])
#y_train = np.array([1 if i==2 else i for i in y_train])

y_test = np.array([-1 if i==0 else i for i in y_test])
#y_test = np.array([0 if i==1 else i for i in y_test])
#y_test = np.array([1 if i==2 else i for i in y_test])
(y_train==-1).sum()

160

In [0]:
dev = qml.device("qulacs.simulator", wires=7,shots=1000,analytic = True)

@qml.qnode(dev)
def qcircuit(params, X=None):
    """A variational quantum circuit representing the Universal classifier.

    Args:
        params (array[float]): array of parameters
        x (array[float]): single input vector
        y (array[float]): single output state density matrix

    Returns:
        float: fidelity between output state and input
    """

    X = np.hstack((X,X))
    qml.templates.AmplitudeEmbedding(X,wires=list(range(7)),pad=0.0,normalize=True,)


    qml.templates.StronglyEntanglingLayers(params,wires=list(range(7)))
  
    
            
    return qml.expval(qml.PauliZ(0))

In [0]:
def cost(params, x, y):
    """Cost function to be minimized.

    Args:
        params (array[float]): array of parameters
        x (array[float]): 2-d array of input vectors
        y (array[float]): 1-d array of targets
    Returns:
        float: loss value to be minimized
    """

    # Compute prediction for each input in data batch
    loss = []
    pred = []
    margin = 0.3
    bias = params[1]

    for i in range(len(x)):

      out =  qcircuit(params[0],X=x[i])+bias

      loss.append((y[i]-out)**2)
    weights = compute_sample_weight('balanced',y)
    s = 0
    for x, y in zip(loss, weights):
      s += x * y

    return s/sum(weights)

In [56]:
np.random.seed(0)
weights = qml.init.strong_ent_layers_uniform(n_layers=6,n_wires=7,high=1,seed=3)
control_weights = np.random.uniform(low=-0.0001,high=.0001,size=(1))
params = [weights,control_weights]
params

[array([[[0.5507979 , 0.70814782, 0.29090474],
         [0.51082761, 0.89294695, 0.89629309],
         [0.12558531, 0.20724288, 0.0514672 ],
         [0.44080984, 0.02987621, 0.45683322],
         [0.64914405, 0.27848728, 0.6762549 ],
         [0.59086282, 0.02398188, 0.55885409],
         [0.25925245, 0.4151012 , 0.28352508]],
 
        [[0.69313792, 0.44045372, 0.15686774],
         [0.54464902, 0.78031476, 0.30636353],
         [0.22195788, 0.38797126, 0.93638365],
         [0.97599542, 0.67238368, 0.90283411],
         [0.84575087, 0.37799404, 0.09221701],
         [0.6534109 , 0.55784076, 0.36156476],
         [0.2250545 , 0.40651992, 0.46894025]],
 
        [[0.26923558, 0.29179277, 0.4576864 ],
         [0.86053391, 0.5862529 , 0.28348786],
         [0.27797751, 0.45462208, 0.20541034],
         [0.20137871, 0.51403506, 0.08722937],
         [0.48358553, 0.36217621, 0.70768662],
         [0.74674622, 0.69109292, 0.68918041],
         [0.37360012, 0.6681348 , 0.33984866]],
 
    

In [0]:
def iterate_minibatches(inputs, targets, batch_size):
    """
    A generator for batches of the input data

    Args:
        inputs (array[float]): input data
        targets (array[float]): targets

    Returns:
        inputs (array[float]): one batch of input data of length `batch_size`
        targets (array[float]): one batch of targets of length `batch_size`
    """
    for id, start_idx in enumerate(range(0, inputs.shape[0] - batch_size + 1, batch_size)):
        idxs = slice(start_idx, start_idx + batch_size)
        yield id,inputs[idxs], targets[idxs]

In [58]:

learning_rate = 0.1
epochs = 1200
batch_size = 25
from pennylane.optimize import NesterovMomentumOptimizer as no
from pennylane.optimize import AdamOptimizer
#opt = AdamOptimizer(learning_rate)

opt = no(learning_rate)
opt.reset()

grads = []
from sklearn.utils import shuffle
from tqdm import tqdm
for it in range(epochs):
    final = []
    prob = []
    X_train_1,y_train_1 = shuffle(X_train,y_train)
    X_test_1,y_test_1 = shuffle(X_test,y_test)

    for id,Xbatch, ybatch in iterate_minibatches(X_train_1, y_train_1, batch_size=batch_size):
        if id == 1:
            break
        params = opt.step(lambda v: cost(v, Xbatch, ybatch), params)
#        loss = cost(params, X_test, y_test, state_labels)
        
        grads.append(params)
        
    if it % 1 == 0:
      #predictions = [variational_classifier(params, x) for x in X_test]

      #predictions = [-1 if (np.sign(i) == -1)  else i for i in predictions]
 #     for i in range(len(X_test)):
#        res = qcircuit(params,X=X_test[i],y=y_test[i])
  #      if y_test[i] == 2 and np.sign(res) ==-1:
   #       final.append()

    #  loss = accuracy_score(y_test,predictions)
      test_loss = cost(params, X_test, y_test)
#        loss = cost(params, X_test, y_test, state_labels)
        #predictions = [np.sign(variational_classifier(params, x=x)) for x in X_test]
        #loss = accuracy_score(y_test,predictions)
      
      print('test accuracy = ',test_loss, " | ","epoch: ", it+1)

test accuracy =  [0.97548368]  |  epoch:  1
test accuracy =  [0.93808834]  |  epoch:  2
test accuracy =  [0.89372963]  |  epoch:  3
test accuracy =  [0.84744184]  |  epoch:  4
test accuracy =  [0.80310297]  |  epoch:  5
test accuracy =  [0.76483756]  |  epoch:  6
test accuracy =  [0.73231219]  |  epoch:  7
test accuracy =  [0.70433238]  |  epoch:  8
test accuracy =  [0.67825435]  |  epoch:  9
test accuracy =  [0.65387167]  |  epoch:  10
test accuracy =  [0.63026455]  |  epoch:  11
test accuracy =  [0.60772851]  |  epoch:  12
test accuracy =  [0.58785215]  |  epoch:  13
test accuracy =  [0.57078943]  |  epoch:  14
test accuracy =  [0.55669051]  |  epoch:  15
test accuracy =  [0.5455617]  |  epoch:  16
test accuracy =  [0.53705242]  |  epoch:  17
test accuracy =  [0.5300163]  |  epoch:  18
test accuracy =  [0.52454971]  |  epoch:  19
test accuracy =  [0.51976112]  |  epoch:  20
test accuracy =  [0.5156373]  |  epoch:  21
test accuracy =  [0.51168409]  |  epoch:  22
test accuracy =  [0.50

KeyboardInterrupt: ignored

In [0]:
def predict(params,x,y,margin):
  final = []
  prob = []
  for i in range(len(x)):
    out = qcircuit(params[0],X=x[i])+params[1]
    prob.append(out)
#    print(out)
    #if -margin < out <= margin:
    #  final.append(0)
    if out < margin:
      final.append(-1)
    if out > margin:
      final.append(1)
#  print(final)
  return accuracy_score(y,final),prob

In [80]:
predict(grads[14],X_test,y_test,margin=-0.1),y_test

((1.0,
  [array([-0.22380664]),
   array([-0.22116113]),
   array([-0.30126537]),
   array([0.23917539]),
   array([-0.31754225]),
   array([-0.37722629]),
   array([0.29926922]),
   array([0.11420272]),
   array([0.26063584]),
   array([-0.25591998]),
   array([-0.34262169]),
   array([0.28932533]),
   array([0.26559402]),
   array([0.32076576]),
   array([0.22833583]),
   array([-0.13998179]),
   array([0.30538435]),
   array([-0.29778336]),
   array([-0.23778385]),
   array([-0.20183471]),
   array([-0.26783934]),
   array([0.09981158]),
   array([0.17459887]),
   array([-0.27013575]),
   array([0.07243808]),
   array([-0.18349443]),
   array([0.25448586]),
   array([-0.27142048]),
   array([0.3518442]),
   array([0.29784298]),
   array([0.17096639]),
   array([-0.40620454]),
   array([-0.26825863]),
   array([-0.28908245]),
   array([0.3676674]),
   array([0.29239058])]),
 array([-1, -1, -1,  1, -1, -1,  1,  1,  1, -1, -1,  1,  1,  1,  1, -1,  1,
        -1, -1, -1, -1,  1,  1, -1,

In [81]:
params

[array([[[ 0.65916659,  0.75269909,  0.462421  ],
         [ 0.79562387,  0.67706819,  1.09141816],
         [ 0.04269267,  0.01407959,  0.00530049],
         [ 0.50310918, -0.03199938,  0.52721615],
         [ 0.7985506 , -0.08636548,  0.82500634],
         [ 0.4203359 , -0.02018885,  0.4194102 ],
         [ 0.17873786,  0.88623972,  0.24192472]],
 
        [[ 1.01789658,  0.64067963, -0.04729443],
         [ 0.17989108,  0.46320836,  0.4830478 ],
         [-0.02629961,  0.60559014,  1.31509806],
         [ 1.31417028,  0.87281993,  0.8555992 ],
         [ 1.18587069,  0.00936877,  0.26065989],
         [ 0.9395481 ,  0.49657997,  0.32993579],
         [ 0.47093586,  0.08285624,  0.5310718 ]],
 
        [[ 0.20126419,  0.7434978 ,  1.00450437],
         [ 0.76657152,  0.61427208,  0.26490833],
         [ 0.38736055,  0.6243423 ,  0.47433956],
         [ 0.2810925 ,  1.25328174,  0.3352466 ],
         [ 0.48472553,  0.25216341,  0.70426291],
         [ 0.66765944,  0.65393333,  0.91235